# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [2]:
!pip install --quiet transformers

^C
ERROR: Operation cancelled by user


Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm.notebook import tqdm
import torchvision
import subprocess
import os

torchvision.disable_beta_transforms_warning()
result = subprocess.run(
    'bash -c "source /etc/network_turbo && env | grep proxy"',
    shell=True,
    capture_output=True,
    text=True
)

output = result.stdout

for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value  

import os

os.environ["HF_HOME"] = "/root/.cache/huggingface"  
os.environ["TRANSFORMERS_OFFLINE"] = "0"            
os.environ["HF_HUB_URL"] = "https://hf-mirror.com"  


Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [5]:

MODEL_NAME = "TurkuNLP/gpt3-finnish-large"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, mirror="https://hf-mirror.com")
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, mirror="https://hf-mirror.com")


Instantiate a text generation pipeline using the tokenizer and model.

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(next(model.parameters()).device)

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

cuda:0


We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [7]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Hyvin, kiitos.”\n”Kiva kuulla.”\n”Kuule, minulla on sinulle asiaa.”\n'}]


Just print the text

In [8]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [9]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee? Ja mä kans haluun sulle jonkun tollasen.. En voi vastustaa.. 🙂 tyksisin kyl ite.. 🙂 hei en saanu yhteyttä tänään


## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [15]:
generation_kwargs = {
    "max_new_tokens": 50,  
    "num_return_sequences": 1,
    "temperature": 0.7,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "truncation": True
}

def generate_response(prompt):
    try:
        result = pipe(prompt, **generation_kwargs)
        generated_text = result[0]['generated_text']
        response = generated_text[len(prompt):].strip()
        return response
    except Exception as e:
        return f"Error: {str(e)}"

prompts = {
    "Zero-shot sentiment": """
Do the following texts express a positive or negative sentiment?
Answer only with "positive" or "negative".

Text: I really enjoyed this movie, it was fantastic!
Answer:
""",
    "One-shot names": """
List the person names occurring in the following texts.
Answer only with the names, separated by commas.

Text: Peter and Susan are siblings.
Answer: Peter, Susan

Text: Mary went to the park with John.
Answer:
""",
    "Two-shot math": """
This is a first grade math exam.
Answer only with the number.

Question: 13 + 24 =
Answer: 37

Question: 42 + 18 =
Answer: 60

Question: 11 + 22 =
Answer:
"""
}
for title, prompt in prompts.items():
    print("="*50)
    print(title)
    print("Prompt:\n", prompt)
    print("Response:", generate_response(prompt))


Zero-shot sentiment
Prompt:
 
Do the following texts express a positive or negative sentiment?
Answer only with "positive" or "negative".

Text: I really enjoyed this movie, it was fantastic!
Answer:

Response: I really enjoyed this movie, it was fantastic!
Text: I really enjoyed this movie, it was fantastic!
Answer:
I really enjoyed this movie, it was fantastic!
Text: I really
One-shot names
Prompt:
 
List the person names occurring in the following texts.
Answer only with the names, separated by commas.

Text: Peter and Susan are siblings.
Answer: Peter, Susan

Text: Mary went to the park with John.
Answer:

Response: In the beginning of the trip, Mary had a phone call from his parents -
“I need to get a better picture of her.”
Answer:
"I have to go and take some photos.
Two-shot math
Prompt:
 
This is a first grade math exam.
Answer only with the number.

Question: 13 + 24 =
Answer: 37

Question: 42 + 18 =
Answer: 60

Question: 11 + 22 =
Answer:

Response: 68

Answer: 42 + 27 =
Answ

**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**